In [256]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

a = pd.read_parquet('ammonium_2024.parquet')
n = pd.read_parquet('nitrate_2024.parquet')
oa = pd.read_parquet('oxygen_a_2024.parquet')
ob = pd.read_parquet('oxygen_b_2024.parquet')
p = pd.read_parquet('phosphate_2024.parquet')
ifd = pd.read_parquet('Influent_Flow_2023.parquet')
edf = pd.read_csv('ExternalDataFinal.csv')


In [257]:
def format_datetime_columns(df, date_cols=['datumBeginMeting', 'datumEindeMeting']):
    """
    Convert specified columns in df to format 'D-M-YYYY HH:MM', dropping leading zeros 
    in day and month. Modifies df in place.
    """
    for col in date_cols:
        df[col] = pd.to_datetime(df[col])
        df[col] = df[col].apply(lambda ts: 
            f"{ts.day}-{ts.month}-{ts.year} {ts.hour:02d}:{ts.minute:02d}"
            if pd.notnull(ts) else None
        )
    return df

for df in [a, n, oa, ob, p, ifd]:
    df.drop(columns=['historianTagnummer', 'waardebewerkingsmethodeCode'], inplace=True)

In [258]:
ifd

,hstWaarde,datumBeginMeting,datumEindeMeting
0,3330.199,2023-01-01 00:00:00,2023-01-01 00:01:00
1,3288.798,2023-01-01 00:01:00,2023-01-01 00:02:00
2,3260.545,2023-01-01 00:02:00,2023-01-01 00:03:00
3,3253.279,2023-01-01 00:03:00,2023-01-01 00:04:00
4,3263.276,2023-01-01 00:04:00,2023-01-01 00:05:00
...,...,...,...
44635,627.162,2023-12-31 23:55:00,2023-12-31 23:56:00
44636,1433.649,2023-12-31 23:56:00,2023-12-31 23:57:00
44637,1637.519,2023-12-31 23:57:00,2023-12-31 23:58:00
44638,1933.898,2023-12-31 23:58:00,2023-12-31 23:59:00


In [259]:
a.rename(columns={'hstWaarde': 'Ammonium mg/L'}, inplace=True)
n.rename(columns={'hstWaarde': 'Nitrate mg/L'}, inplace=True)
p.rename(columns={'hstWaarde': 'Phosphate mgP/L'}, inplace=True)
oa.rename(columns={'hstWaarde': 'Oxygen_A mg/L'}, inplace=True)
ob.rename(columns={'hstWaarde': 'Oxygen_B mg/L'}, inplace=True)
ifd.rename(columns={'hstWaarde': 'influentFlow'}, inplace=True)

In [260]:
a = a.rename(columns={'datumEindeMeting': 'datumEindeMeting_A'})
n = n.rename(columns={'datumEindeMeting': 'datumEindeMeting_N'})
p = p.rename(columns={'datumEindeMeting': 'datumEindeMeting_P'})
oa = oa.rename(columns={'datumEindeMeting': 'datumEindeMeting_Oa'})
ob = ob.rename(columns={'datumEindeMeting': 'datumEindeMeting_Ob'})

In [261]:
a = a.drop_duplicates(subset='datumEindeMeting_A')
n = n.drop_duplicates(subset='datumEindeMeting_N')
p = p.drop_duplicates(subset='datumEindeMeting_P')
oa = oa.drop_duplicates(subset='datumEindeMeting_Oa')
ob = ob.drop_duplicates(subset='datumEindeMeting_Ob')

In [262]:
ifd = ifd.merge(a[['datumEindeMeting_A', 'Ammonium mg/L']], left_on='datumEindeMeting', right_on='datumEindeMeting_A', how='left')
ifd = ifd.merge(n[['datumEindeMeting_N', 'Nitrate mg/L']], left_on='datumEindeMeting', right_on='datumEindeMeting_N', how='left')
ifd = ifd.merge(p[['datumEindeMeting_P', 'Phosphate mgP/L']], left_on='datumEindeMeting', right_on='datumEindeMeting_P', how='left')
ifd = ifd.merge(oa[['datumEindeMeting_Oa', 'Oxygen_A mg/L']], left_on='datumEindeMeting', right_on='datumEindeMeting_Oa', how='left')
ifd = ifd.merge(ob[['datumEindeMeting_Ob', 'Oxygen_B mg/L']], left_on='datumEindeMeting', right_on='datumEindeMeting_Ob', how='left')


In [263]:
ifd = ifd.drop(columns=['datumEindeMeting_A', 'datumEindeMeting_N', 'datumEindeMeting_P', 'datumEindeMeting_Oa', 'datumEindeMeting_Ob'])

In [264]:
ifd = ifd.drop_duplicates()

ifd.reset_index(drop=True, inplace=True)

# External Data


In [266]:
edf.head(3)

,station_code,date,hour,T,T10N,TD,DR,RH,N,M,R,S,O,Y,datumBeginMeting
0,215,2023-01-01 00:00:00+00:00,1,142,NaN,84,0,0,8.0,0,0,0,0,0,2023-01-01 01:00:00
1,215,2023-01-01 00:00:00+00:00,2,144,NaN,85,0,0,8.0,0,0,0,0,0,2023-01-01 02:00:00
2,215,2023-01-01 00:00:00+00:00,3,139,NaN,82,0,0,8.0,0,0,0,0,0,2023-01-01 03:00:00


In [267]:
ifd.head(3)

,influentFlow,datumBeginMeting,datumEindeMeting,Ammonium mg/L,Nitrate mg/L,Phosphate mgP/L,Oxygen_A mg/L,Oxygen_B mg/L
0,3330.199,2023-01-01 00:00:00,2023-01-01 00:01:00,1.31,4.73,0.001,1.663,0.777
1,3288.798,2023-01-01 00:01:00,2023-01-01 00:02:00,1.251,4.73,0.001,1.789,0.982
2,3260.545,2023-01-01 00:02:00,2023-01-01 00:03:00,1.182,4.73,0,2.105,1.189


# New combined dataset name is: merged

In [269]:
# merge ifd(2024 data) and external data
merged = pd.merge(ifd, edf, on='datumBeginMeting', how='outer')

In [270]:
merged.drop_duplicates(inplace=True)


In [271]:
merged.head(65)

,influentFlow,datumBeginMeting,datumEindeMeting,Ammonium mg/L,Nitrate mg/L,Phosphate mgP/L,Oxygen_A mg/L,Oxygen_B mg/L,station_code,date,...,T10N,TD,DR,RH,N,M,R,S,O,Y
0,3330.199,2023-01-01 00:00:00,2023-01-01 00:01:00,1.31,4.73,0.001,1.663,0.777,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3288.798,2023-01-01 00:01:00,2023-01-01 00:02:00,1.251,4.73,0.001,1.789,0.982,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3260.545,2023-01-01 00:02:00,2023-01-01 00:03:00,1.182,4.73,0,2.105,1.189,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3253.279,2023-01-01 00:03:00,2023-01-01 00:04:00,1.181,4.675,0,2.271,1.266,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3263.276,2023-01-01 00:04:00,2023-01-01 00:05:00,1.179,4.605,0,2.168,1.403,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,2916.007,2023-01-01 01:00:00,2023-01-01 01:01:00,1.119,3.616,0.01,0.012,0.096,215.0,2023-01-01 00:00:00+00:00,...,NaN,84.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0
61,2916.007,2023-01-01 01:00:00,2023-01-01 01:01:00,1.119,3.616,0.01,0.012,0.096,290.0,2023-01-01 00:00:00+00:00,...,NaN,72.0,0.0,-1.0,8.0,0.0,1.0,0.0,0.0,0.0
62,2903.295,2023-01-01 01:01:00,2023-01-01 01:02:00,1.13,3.595,0.017,0.013,0.097,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,2838.847,2023-01-01 01:02:00,2023-01-01 01:03:00,1.141,3.575,0.025,0.014,0.097,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


T: Temperature (in 0.1 degrees Celsius) at 1.50 m height during the observation

T10N: Minimum temperature (in 0.1 degrees Celsius) at 10 cm height in the last 6 hours

TD: Dew point temperature (in 0.1 degrees Celsius) at 1.50 m height during the observation

DR: Duration of precipitation (in 0.1 hours) per hour period

RH: Hourly sum of precipitation (in 0.1 mm) (-1 for <0.05 mm)
                                             
N: Cloud cover (upper air coverage in eighths), during observation (9=upper air invisible)

M: Fog 0=did not occur, 1=did occur in the previous hour and/or during the observation

R: Rain 0=did not occur, 1=did occur in the previous hour and/or during the observation

S: Snow 0=not occurred, 1=occurred in the previous hour and/or during the observation

O: Thunderstorm 0=did not occur, 1=did occur in the previous hour and/or during the observation

Y: Ice formation 0=not occurred, 1=occurred in the previous hour and/or during the observation